In [1]:
import os
import logging
import struct
import random
import math
import numpy as np
import pandas as pd
import tensorflow as tf
import pydot
import pydotplus
import graphviz
import scipy
import scipy.stats as sts
import numpy.random as npr
from logging.config import dictConfig
from scipy.stats import norm
from sklearn import metrics
from keras import regularizers
from keras.layers import Dense, Activation, Input, Dropout, BatchNormalization
from keras.models import Sequential
from keras_tqdm import TQDMNotebookCallback as ktqdm
from tqdm import tqdm_notebook as tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from statsmodels.distributions.empirical_distribution import ECDF
from matplotlib import pyplot as plt
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.metrics import precision_recall_curve, average_precision_score, confusion_matrix

%matplotlib notebook

Using TensorFlow backend.


## Network Metrics Function

In [2]:
def plot_history_graph(history, index, label, name):
    """Plot the graph for the given history metrics."""
    plt.plot(history.history[index], label='Training %s = %0.6f' % (label, history.history[index][-1]))
    plt.plot(history.history['val_%s'%index] , label='Testing %s = %0.6f' % (label, history.history['val_%s'%index][-1]))
    plt.title('Model %s'%label, fontsize=15)
    plt.ylabel(label)
    plt.xlabel('epoch')
    plt.legend(loc='upper left')
    plt.savefig(name, bbox_inches=None)
    plt.show()
    #plt.figure(figsize=(80,40))

## Custom Activation Function

In [3]:
from keras import backend as K
from keras.utils.generic_utils import get_custom_objects

beta005 = 0.05
beta02 = 0.2
beta05 = 0.5
beta1 = 1
beta3 = 3
beta10 = 10

def custom_activation(x):
    return (K.sigmoid(x))

def custom_activation005(x):
    return (K.sigmoid(x * 0.05))

def custom_activation02(x):
    return (K.sigmoid(x * 0.2))

def custom_activation05(x):
    return (K.sigmoid(x * 0.5))

def custom_activation3(x):
    return (K.sigmoid(x * 3))

def custom_activation10(x):
    return (K.sigmoid(x * 10))

## Reproducibility

In [4]:
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(42)
random.seed(42)
tf.set_random_seed(42)

## Discrete Uniform

In [5]:
step = 50
size = 10000
n = step * size
train_size = 0.70
test_size = 1 - train_size
width = np.ceil(np.log2(n)).astype(int)
predecessors = np.arange(0, n, step)
predecessors = np.array([
    (predecessors[i]) for i in range(size)
]).astype(int)
predecessors, predecessors.shape, n

(array([     0,     50,    100, ..., 499850, 499900, 499950]),
 (10000,),
 500000)

In [6]:
cumulative_predecessors = np.array([(((i / size))) for i in range(size)])

cumulative_predecessors, cumulative_predecessors.size

(array([0.000e+00, 1.000e-04, 2.000e-04, ..., 9.997e-01, 9.998e-01,
        9.999e-01]), 10000)

In [7]:
predecessors_bits = np.array([
    list(np.binary_repr(i, width)) for i in np.arange(0, n, step)
]).astype(int)
np.set_printoptions(threshold=100)
predecessors_bits, predecessors_bits.shape

(array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 1, 0],
        [0, 0, 0, ..., 1, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 1, 0],
        [1, 1, 1, ..., 1, 0, 0],
        [1, 1, 1, ..., 1, 1, 0]]), (10000, 19))

In [8]:
## Create training + testing set
## L'idea è utilizzare una permutazione casuale per ottenere gli elementi presenti da 0 a size
rnd_ind = npr.permutation(size)
rnd_ind

array([6252, 4684, 1731, ..., 5390,  860, 7270])

In [9]:
permutated_preds, permutated_cums = predecessors[np.ndarray.tolist(rnd_ind)], cumulative_predecessors[np.ndarray.tolist(rnd_ind)]
train_ind, train_lab = permutated_preds[:int(size * 0.70)], permutated_cums[:int(size * 0.70)]
test_ind, test_lab = permutated_preds[int(size * 0.70):], permutated_cums[int(size * 0.70):]
permutated_preds, permutated_cums

(array([312600, 234200,  86550, ..., 269500,  43000, 363500]),
 array([0.6252, 0.4684, 0.1731, ..., 0.539 , 0.086 , 0.727 ]))

In [10]:
train_ind_2 = np.array([list(np.binary_repr(train_ind[i], width)) for i in range(train_ind.size)]).astype(int)
train_ind_2

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0, 0],
       [0, 0, 1, ..., 1, 1, 0],
       ...,
       [0, 1, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [1, 0, 0, ..., 1, 1, 0]])

In [11]:
test_ind_2 = np.array([list(np.binary_repr(test_ind[i], width)) for i in range(test_ind.size)]).astype(int)
test_ind_2

array([[1, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 1, 1, 0],
       [1, 0, 1, ..., 1, 1, 0],
       ...,
       [1, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 1, ..., 1, 0, 0]])

## Model building

In [12]:
def build_model(neurons):
    """ Return keras network model """
    model = Sequential()
    model.add(Dense(int(neurons * 3), input_dim=neurons, name="Input_dense_layer", activation=custom_activation10))
    model.add(BatchNormalization())
    model.add(Dense(int(neurons * 2), name="Hidden_dense_layer", activation="sigmoid"))
    model.add(BatchNormalization())
    model.add(Dense(1, name="Output_dense_layer", activation="sigmoid"))
    model.compile(
        loss='mean_squared_error', 
        optimizer="adam",
        metrics=['mean_absolute_error'])

    return model

In [13]:
from keras.utils import plot_model

def cross_val(neurons, x, y):
    n_folds = 5
    valutations = []
    sizex = x.shape[0]
    sizey = y.shape[0]
    x_1, y_1 = x[:int(sizex * 0.20)], y[:int(sizey * 0.20)]
    x_2, y_2 = x[(int(sizex * 0.20)) : (int(sizex * 0.40))], y[(int(sizey * 0.20)) : (int(sizey * 0.40))]
    x_3, y_3 = x[(int(sizex * 0.40)) : (int(sizex * 0.60))], y[(int(sizey * 0.40)) : (int(sizey * 0.60))]
    x_4, y_4 = x[(int(sizex * 0.60)) : (int(sizex * 0.80))], y[(int(sizey * 0.60)) : (int(sizey * 0.80))]
    x_5, y_5 = x[int(sizex * 0.80):], y[int(sizey * 0.80):]
    for i in range(n_folds):
        os.environ['PYTHONHASHSEED'] = '0'
        np.random.seed(42)
        random.seed(42)
        tf.set_random_seed(42)
        print("Fold: ", i, "/", n_folds)
        
        if i == 0:
            x_val, y_val = x_1, y_1
            print("Inside if")
            x_train = np.append(x_2, np.append(x_3, np.append(x_4, x_5, axis = 0), axis= 0), axis=0)
            y_train = np.append(y_2, np.append(y_3, np.append(y_4, y_5, axis = 0), axis= 0), axis=0)
            print("train x and y shape")
        elif i == 1:
            x_val, y_val = x_2, y_2
            x_train = np.append(x_1, np.append(x_3, np.append(x_4, x_5, axis = 0), axis= 0), axis=0)
            y_train = np.append(y_1, np.append(y_3, np.append(y_4, y_5, axis = 0), axis= 0), axis=0)
        elif i == 2:
            x_val, y_val = x_3, y_3
            x_train, y_train = np.append(x_1, [x_2, x_4, x_5]), np.append(y_1, [y_2, y_4, y_5])
            x_train = np.append(x_1, np.append(x_2, np.append(x_4, x_5, axis = 0), axis= 0), axis=0)
            y_train = np.append(y_1, np.append(y_2, np.append(y_4, y_5, axis = 0), axis= 0), axis=0)
        elif i == 3:
            x_val, y_val = x_4, y_4
            x_train = np.append(x_1, np.append(x_2, np.append(x_3, x_5, axis = 0), axis= 0), axis=0)
            y_train = np.append(y_1, np.append(y_2, np.append(y_3, y_5, axis = 0), axis= 0), axis=0)
        else:
            x_val, y_val = x_5, y_5
            x_train = np.append(x_1, np.append(x_2, np.append(x_3, x_4, axis = 0), axis= 0), axis=0)
            y_train = np.append(y_1, np.append(y_2, np.append(y_3, y_4, axis = 0), axis= 0), axis=0)
        ## Part of cross validation ##
        model = build_model(neurons)
        print("Model built")
        history = model.fit(
            x_train, 
            y_train,
            epochs=100,
            shuffle=True,
            batch_size=256,
            verbose=0,
            callbacks=[ktqdm(metric_format="{name}: {value:e}")],
            validation_data=(x_val, y_val)
        )
        print("Model fit")
        valutations = valutations + [model.evaluate(x_val, y_val, steps=100)]
    
    return valutations

In [14]:
vals = cross_val(width, train_ind_2, train_lab)
vals

Fold:  0 / 5
Inside if
train x and y shape
Model built



Model fit
100/100 [==============================] - 2s 16ms/step
Fold:  1 / 5
Model built



Model fit
100/100 [==============================] - 1s 14ms/step
Fold:  2 / 5
Model built



Model fit
100/100 [==============================] - 1s 12ms/step
Fold:  3 / 5
Model built



Model fit
100/100 [==============================] - 1s 12ms/step
Fold:  4 / 5
Model built



Model fit
100/100 [==============================] - 1s 13ms/step


[[0.00038050641887821257, 0.015649951994419098],
 [0.00041276024421676993, 0.016551265493035316],
 [0.00037584599340334535, 0.015610873699188232],
 [0.00030519196297973394, 0.013968709856271744],
 [0.00034076860174536705, 0.014660402201116085]]

In [15]:
mse_mean = (vals[0][0] + vals[1][0] + vals[2][0] + vals[3][0] + vals[4][0]) / 5

In [16]:
mae_mean = (vals[0][1] + vals[1][1] + vals[2][1] + vals[3][1] + vals[4][1]) / 5

In [17]:
mae_mean, mse_mean

(0.015288240648806095, 0.00036301464424468575)

In [18]:
log = open("log_2.txt", "a")
log.writelines(("Beta = 10\n",
                "\tMAE mean:\t", str(mae_mean), "\n",
                "\tMSE mean:\t", str(mse_mean),"\n\n"))
log.close()